### Load selected rows

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/selected/selected.csv')
df.describe()

,roast_amount_kg,1st_malt_amount_kg,2nd_malt_amount_kg,mt_temperature,mt_time,wk_temperature,wk_steam,wk_time,total_cold_wort,ph,extract,color,woc_time,whp_transfer_time,whp_rest_time,1st_malt_color,2nd_malt_color
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,40.364099,13708.801941,6518.586737,67.819929,6755.558000,105.240010,6715.525000,6776.986333,936.898333,5.622935,15.202680,17.741587,3101.783500,752.844000,14.495181,6.048667,6.019283
std,17.982047,1369.917810,1102.770677,2.861199,440.785944,0.636487,279.615828,588.860492,134.627923,0.744504,1.017336,9.645823,548.827876,59.693521,0.007603,0.081353,0.015237
min,17.415013,0.000000,3996.957605,66.866813,6458.600000,97.659771,6170.500000,5929.700000,68.800000,2.295564,6.796530,11.542656,107.500000,719.175000,14.476667,5.912500,5.912500
25%,24.778737,13944.859845,6068.565933,66.909513,6620.387500,105.202896,6568.250000,6554.275000,954.600000,5.547247,15.148521,12.892941,3123.950000,729.925000,14.494583,6.020000,6.020000
50%,28.272487,14041.300940,6074.067303,67.002759,6700.475000,105.269124,6654.250000,6665.000000,963.200000,5.625450,15.391154,13.418677,3143.837500,736.375000,14.494583,6.020000,6.020000
75%,64.177513,14087.302593,6160.225824,67.208414,6773.575000,105.373703,6802.062500,6807.437500,968.575000,5.743791,15.551699,14.864908,3171.250000,743.900000,14.494583,6.127500,6.020000
max,88.042526,16092.750000,8434.826355,81.452861,11671.275000,105.718793,8202.250000,12621.575000,1088.975000,7.382991,16.320075,44.990000,4876.200000,1241.625000,14.584167,6.127500,6.127500


In [2]:
import random
from typing import Literal
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import r2_score, mean_absolute_error as mae, mean_squared_error as mse
def baseline_dummy_metrics(df, samples, strategy:Literal['median','mean']):
    '''estimate baseline values for dataset using sklearn dummy regressor
    samples is the number of fits to be done

    returns
    -------
    df with r2_score, mean absolute error, mean squared error and std deviations respectively
    avreage for every fit 
    '''
    values = pd.DataFrame()
    for i in range(samples):
        y = df['color']
        X = df.drop('color', axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random.randint(1,1000))

        dummy_regr = DummyRegressor(strategy=strategy)
        dummy_regr.fit(X_train, y_train)
        
        y_pred = dummy_regr.predict(X_test)
        values = pd.concat([values, pd.DataFrame.from_records({'r2_score':[r2_score(y_test,y_pred)], 'mae':[mae(y_test,y_pred)], 'mse':[mse(y_test,y_pred)]})], ignore_index=True)
    _dict = {}
    for col in values.columns:
        _dict[col] = values[col].mean()
        _dict[f'{col}_std'] = values[col].std()
    return pd.DataFrame.from_records([_dict])

baseline = baseline_dummy_metrics(df, 10000, 'median')
print('Baseline values for metrics')
baseline

Baseline values for metrics


,mae,mae_std,mse,mse_std,r2_score,r2_score_std
0,4.914964,1.096135,110.335714,32.996268,-0.205678,0.065793
